In [1]:
import re

import xlwings as xw
import pandas as pd
from shapely.wkt import loads
from win32com.universal import com_error

from src.parsing import Splitter, Parser, Templates

In [2]:
sheet_taxation_list = xw.sheets['Ведомость']
taxation_list_df = sheet_taxation_list.range('A1').expand().options(pd.DataFrame, header=1).value
taxation_list_df = taxation_list_df[
    ['Номер точки', 'Наименование', 'Количество', 'Высота', 'Толщина', 'Состояние', 'Кустарник']]

taxation_list_df

,Номер точки,Наименование,Количество,Высота,Толщина,Состояние,Кустарник
Индекс,,,,,,,
1.0,1,Берёза пушистая,1,14,0.35,Хорошее,0.0
2.0,2,Берёза пушистая,1,14,0.35,Хорошее,0.0
3.0,3,Берёза пушистая,1,14,0.35,Хорошее,0.0
4.0,4,Берёза пушистая,1,13,0.30,Хорошее,0.0
5.0,5,Берёза пушистая,1,0.2,0.32,Хорошее,0.0
...,...,...,...,...,...,...,...
180.0,180,Клён ясенелистный,1,0.2,0.07,Хорошее,0.0
181.0,181,Берёза пушистая,2 ствола,0.2х2,"0.08, 0.07",Хорошее,0.0
182.0,182,Берёза пушистая,1,0.2,0.04,Хорошее,0.0


In [3]:
def get_shapes_from_autocad_df(df: pd.DataFrame, number: str) -> dict:
    """
    Получение геометрии из датафрейма "Автокад" по номеру точки
    Args:
        df (pd.DataFrame): Датафрейм "Автокад"
        number (str): Номер точки

    Returns:
        dict: Словарь двух колонок для датафрейма со списками позиций номеров и геометрий
    """
    number_positions, geometries = [], []
    split_numbers = Splitter.number(number)
    df = df.set_index('Разделенный номер')
    for split_number in split_numbers:
        shapes = df.loc[split_number][['Позиция номера', 'Геометрия']].to_dict()
        number_positions.append(shapes['Позиция номера'])
        geometries.append(shapes['Геометрия'])
    return {'Список позиций номеров': number_positions, 'Список геометрии': geometries}

def split_taxation_list_item(df: pd.DataFrame, series: pd.Series) -> list[dict]:
    """
    Получение строки ОРМ из строки таблицы Ведомости

    Args:
        df (pd.DataFrame): Датафрейм листа Автокад
        series (pd.Series): Строка таблицы

    Returns:
        list[dict]: Список словарей ОРМ для датафрейма
    """
    match_trunk = re.search(Templates.TRUNKS, series['Количество'])
    match_contour = re.search(Templates.CONTOUR, series['Количество'])
    match_line = re.search(Templates.LINE, series['Количество'])
    shapes = get_shapes_from_autocad_df(df, series['Номер точки'])
    numbers_positions, geometries = shapes['Список позиций номеров'], shapes['Список геометрии']
    if not match_contour and not match_line and not match_trunk:
        split_numbers = Splitter.number(series['Номер точки'])
        split_height = Splitter.size(series['Высота'])
        split_diameter = Splitter.size(series['Толщина'])
        split_quality = Splitter.quality(series['Состояние'])
        is_stump = Parser.identification_stump(series['Высота'], series['Толщина'], bool(series['Кустарник']))
        if (not is_stump or "пень" in series['Наименование'].lower()) and len(split_quality) == 1 and len(
                numbers_positions) == 1 and len(geometries) == 1:
            series_dict = series.to_dict()
            series_dict['Позиция номера'] = numbers_positions[0]
            series_dict['Геометрия'] = geometries[0]
            return [series_dict]
        if len(split_height) == 1:
            split_height = split_height * int(series['Количество'])
        if len(split_diameter) == 1:
            split_diameter = split_diameter * int(series['Количество'])
        if len(split_quality) == 1:
            split_quality = split_quality * int(series['Количество'])
        series_data = []
        for idx in range(len(split_numbers)):
            if "пень" not in series['Наименование'].lower():
                is_stump = Parser.identification_stump(split_height[idx], split_diameter[idx],
                                                       bool(series['Кустарник']))
                name = series['Наименование'] + " (пень)" if is_stump else series['Наименование']
            else:
                name = series['Наименование']
            series_data.append({
                'Номер точки': split_numbers[idx],
                'Наименование': name,
                'Количество': 1,
                'Высота': split_height[idx],
                'Толщина': split_diameter[idx],
                'Состояние': split_quality[idx],
                'Кустарник': series['Кустарник'],
                'Позиция номера': numbers_positions[idx],
                'Геометрия': geometries[idx]
            })
        return series_data
    else:
        series_dict = series.to_dict()
        series_dict['Позиция номера'] = numbers_positions[0]
        series_dict['Геометрия'] = geometries[0]
        return [series_dict]

In [4]:
sheet_autocad = xw.sheets['Автокад']
autocad_df = sheet_autocad.range('A1').expand().options(pd.DataFrame, header=1, index=False).value
autocad_df['Позиция номера'] = autocad_df['Позиция номера'].apply(lambda x: loads(x))
autocad_df['Геометрия'] = autocad_df['Геометрия'].apply(lambda x: loads(x))

assert autocad_df['Разделенный номер'].is_unique

# autocad_df

In [5]:
taxation_list_orm = []
for _, series in taxation_list_df.iterrows():
    taxation_list_orm.extend(split_taxation_list_item(autocad_df, series))

taxation_list_orm_df = pd.DataFrame(taxation_list_orm)

taxation_list_orm_df

,Номер точки,Наименование,Количество,Высота,Толщина,Состояние,Кустарник,Позиция номера,Геометрия
0,1,Берёза пушистая,1,14,0.35,Хорошее,0.0,POINT (1970.966646588104 3716.1907815376394),POINT (1970.966646588104 3716.1907815376394)
1,2,Берёза пушистая,1,14,0.35,Хорошее,0.0,POINT (1968.1334147878463 3715.990801322688),POINT (1968.1334147878463 3715.990801322688)
2,3,Берёза пушистая,1,14,0.35,Хорошее,0.0,POINT (1966.2543365259978 3715.5751353734922),POINT (1966.2543365259978 3715.5751353734922)
3,4,Берёза пушистая,1,13,0.30,Хорошее,0.0,POINT (1952.0517949876419 3724.083765277848),POINT (1952.0517949876419 3724.083765277848)
4,5,Берёза пушистая (пень),1,0.2,0.32,Хорошее,0.0,POINT (1951.1303078597275 3725.639428001092),POINT (1951.1303078597275 3725.639428001092)
...,...,...,...,...,...,...,...,...,...
179,180,Клён ясенелистный (пень),1,0.2,0.07,Хорошее,0.0,POINT (1894.343055997487 3891.5020170772855),POINT (1894.343055997487 3891.5020170772855)
180,181,Берёза пушистая,2 ствола,0.2х2,"0.08, 0.07",Хорошее,0.0,POINT (1892.7149838028663 3890.925740151263),POINT (1892.7149838028663 3890.925740151263)
181,182,Берёза пушистая (пень),1,0.2,0.04,Хорошее,0.0,POINT (1891.5655851948059 3889.291014087694),POINT (1891.5655851948059 3889.291014087694)
182,183,Клён ясенелистный,6 стволов,0.2х6,"0.06х3, 0.03, 0.02х2",Хорошее,0.0,POINT (1888.5263379687174 3887.6149663456576),POINT (1888.5263379687174 3887.6149663456576)
